In [ ]:
import os
import sys
import yaml
import time
import pickle
import random
import argparse
import feedparser
import urllib.request
from termcolor import colored
from utils import Config, safe_pickle_dump
flatten = lambda l: [item for sublist in l for item in sublist]

In [ ]:
db = pickle.load(open(Config.db_path, 'rb'))
items = db.items()

In [ ]:
for key, value in db.items():
    db[key]["categories"] = list(map(lambda x: x["term"], value["tags"]))

In [ ]:
ags = [v for k,v in db.items() if 'math.AG' in v["categories"]]
ags[0:1]

In [ ]:
import pandas as pd
s = pd.DataFrame(ags).sort_values("updated",ascending=False)
s[["_rawid", "title", "summary", "author", "published", "updated"]].head(20)

In [ ]:
s[s['title'].str.contains("Stratification of algebraic quotients")][["title", "updated"]]

In [ ]:
import os
import time
import pickle
import shutil
import random
import urllib
import requests
from  urllib.request import urlopen
from termcolor import colored
from utils import Config

printgreen = lambda q: print(colored(q, 'green'))
printred = lambda q: print(colored(q, 'red'))


timeout_secs = 10 # after this many seconds we give up on a paper
if not os.path.exists(Config.pdf_dir): os.makedirs(Config.pdf_dir)
have = set(os.listdir(Config.pdf_dir)) # get list of all pdfs we already have

numok = 0
numtot = 0
db = pickle.load(open(Config.db_path, 'rb'))

session = requests.session()
session.proxies = {
    'http': 'socks5://localhost:9050',
    'https': 'socks5://localhost:9050'
}
headers = {'User-agent': 'HotJava/1.1.2 FCS'}
#p = {'http': 'http://138.186.23.66:53281'}
#proxy = urllib.request.ProxyHandler(p)
#opener = urllib.request.build_opener(proxy)
#urllib.request.install_opener(opener)

start_time = time.time()
num_session = 0
items = db.items()

In [ ]:
pid,j = db.popitem()
print(pid, j)

In [ ]:
pdfs = [x['href'] for x in j['links'] if x['type'] == 'application/pdf']
print(pdfs)
assert len(pdfs) == 1
pdf_url = pdfs[0] + '.pdf'
print(pdf_url)
basename = pdf_url.split('/')[-1]
fname = os.path.join(Config.pdf_dir, basename)
print(fname)

In [ ]:
req = session.get(pdf_url, timeout=15)
open(fname, 'wb').write(req.content)

In [1]:
import os
import time
import pickle
import shutil
import random
import urllib
import requests
import pandas as pd
from utils import Config
import concurrent.futures
from termcolor import colored
from  urllib.request import urlopen

printgreen = lambda q: print(colored(q, 'green'))
printred = lambda q: print(colored(q, 'red'))

timeout_secs = 15
session = requests.session()
session.proxies = {
    'http': 'socks5://localhost:9050',
    'https': 'socks5://localhost:9050'
}
headers = {'User-agent': 'HotJava/1.1.2 FCS'}

db = pickle.load(open(Config.db_path, 'rb'))

numok = 0
numtot = 0

def getpdf(input_item):
    pid, j = input_item
    pdfs = [x['href'] for x in j['links'] if x['type'] == 'application/pdf']
    assert len(pdfs) == 1
    pdf_url = pdfs[0] + '.pdf'
    basename = pdf_url.split('/')[-1]
    fname = os.path.join(Config.pdf_dir, basename)
    if not basename in have:
        req = session.get(pdf_url, timeout=timeout_secs)
        open(fname, 'wb').write(req.content)
        return(1, fname, pdf_url, input_item)
    else:
        return(0, fname, pdf_url, input_item)
printgreen("Setup complete, ready to run.")

Setup complete, ready to run.


In [20]:
paritems = list(db.items())

num_items = len(paritems)
num_processed = 0
num_skipped = 0
num_new = 0
num_symbols = 0
num_batch = 0
num_errors = 0

if not os.path.exists(Config.pdf_dir): 
    os.makedirs(Config.pdf_dir)
    
#os.system("find . -name './data/pdf/*.pdf' -size 0 -print0 | xargs -0 rm")
have = set(os.listdir(Config.pdf_dir)) 

start_time = time.time()

printgreen("Starting. \nDB Size: {} \nFiles Found: {}".format(len(db), len(have)))

def printSummary():
    tot_time = time.time() - start_time
    remaining = len(db) - num_processed
    s = "Papers \nProcessed: {}\nNew: {}\nSkipped: {}\nRemaining: {}"
    printgreen( s.format(num_processed, num_new, num_skipped, remaining) )
    printgreen("Elapsed time: {} seconds".format(tot_time))
    if (num_new > 1):
        printgreen("Proportions: {} new, {} skipped".format( round(num_new/num_processed, 4), round(num_skipped/num_processed, 4) ))
        rate = num_new/tot_time # papers/second
        printgreen("Estimated acquisition rate: {} papers/minute".format(round( (rate) * (60/1) , 2)))
        est_time = remaining/rate # seconds/paper * papers = seconds
        printgreen("Estimated remaining time: {} hours".format(round(float(est_time / (60*60)), 4)))
    else:
        printred("No new papers acquired.")
    print("-------------------------------------------------------------------")
        
with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
    future_to_url = (executor.submit(getpdf, input_item) for input_item in paritems)
    time1 = time.time()
    for future in concurrent.futures.as_completed(future_to_url):
        try:
            status, fname, pdf_url, item = future.result()                
            if status == 1:
                #printgreen("Success: {}".format(fname))
                num_new += 1
                num_batch += 1
                print(colored('!', 'green'), end='')
                num_symbols += 1
            else:
                if(num_skipped % 1000 == 1):
                    print('.', end='')
                    num_symbols += 1
                #print('%s exists, skipping' % (fname, ))
                num_skipped += 1
                
        except Exception as e:
            print(colored('x', 'red'), end='')
            num_errors += 1
            num_symbols += 1
            if(num_errors > 1 and num_errors % 50 == 1):
                print('|')
                #print('\n', e, '\n')
        finally:
            num_processed += 1
            if(num_processed > 1 and num_symbols > 1 and num_symbols % 100 == 1):
                if (num_new > 1 and (num_symbols % 500 == 1 or num_new % 20 == 1)):
                    printSummary()
                    num_batch = 0

printgreen("\n\DONE")

Starting. 
DB Size: 103579 
Files Found: 101558
........!!!!!.!!!!!.!!!!!!!!!!!!!!.!!!!!!!!!!!!!!!!.!!!!!!!!!!!!!!!!!!!!.!!!!!!!!.!!!!!!!!!!!!!!!!.xx!!!!!!!.!x!!!!!!!!.!!!!!!!!!!!x!!!xx.!!!!!!!!!!x!!!!!!!!!!!!!!!!!.!!!!!!!!!!!x!!!!!x!!!x.x!!!!!!!!!!xxxxxxxxxxxxxxxx!!.!xxxxxxx.!xx!!!xxxxxxxx.xxxxxxx|
xxxxxxxxx.xxxxxxxxxxxxxx.xxxxxxxxxxxxxxxxxxx.xxxxxxxx|
xxxxxxxxxxxxxxxxxxxx.xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx|
xxxxxxxx.xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx.x|
xxxxxxxxxxxxxxxxxxxxxxxxxxxxx.xxxxxxxxxxxxxxxxxxxxx|
xxxxxxxxx.xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxPapers 
Processed: 31333
New: 177
Skipped: 30863
Remaining: 72246
Elapsed time: 132.38649892807007 seconds
Proportions: 0.0056 new, 0.985 skipped
Estimated acquisition rate: 80.22 papers/minute
Estimated remaining time: 15.01 hours
-------------------------------------------------------------------
Papers 
Processed: 31334
New: 177
Skipped: 30864
Remaining: 72245
Elapsed time: 132.3873372077942 seconds
Proportions: 0.0056 new, 0.985 skip

xxxxxxxx|
xx.xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx.xxxxxxxxxxxxx|
xxxxxxxxxxxxxxxxxxxxxxxxx.xxxxxxxxxxxxxxxxxxx!!!!!!!!!!!!!!!!!!!xxxxx!.!!!!!!!!!!!!!!!x|
x!xxxx!xxx!x!xxxxx!!!!!!!!!x!!!!!!x!!!xx!!!!!!!xx!.!x!!!!!!!!!!!!!xx!!x!!xx!!!!x.!!!!!!x!!!x!!!!x!!!!!x!!xxx!!!!!!!!!!x!!!!!!!!!!!!!xxxx.xxx!!xxxxxx!!!!xx|
!!!!x!xx!!!!!!x!!xxx.x!xxxxxx!xx!!!x!!!xxxxxx!x!!!!!!!!.!!!!!!xxxxxxx!!!!!!!!!!!!!x!!!!.!!!!!x!!!!xx!!!x!!!x!!!!!!!!x!!!x!!!!!x!!.!!!!!!!!!x!!!!!!!!xxx!x!!xxxxx|
xx.xxxxxxxxxxx!x!xxxxx!x!xx!xx!!!!!.!xxPapers 
Processed: 44024
New: 430
Skipped: 43067
Remaining: 59555
Elapsed time: 376.8330223560333 seconds
Proportions: 0.0098 new, 0.9783 skipped
Estimated acquisition rate: 68.47 papers/minute
Estimated remaining time: 14.4976 hours
-------------------------------------------------------------------
Papers 
Processed: 44025
New: 430
Skipped: 43068
Remaining: 59554
Elapsed time: 376.833208322525 seconds
Proportions: 0.0098 new, 0.9783 skipped
Estimated acquisition rate: 68.47 papers/mi

Estimated remaining time: 14.4919 hours
-------------------------------------------------------------------
Papers 
Processed: 44049
New: 430
Skipped: 43092
Remaining: 59530
Elapsed time: 377.69341945648193 seconds
Proportions: 0.0098 new, 0.9783 skipped
Estimated acquisition rate: 68.31 papers/minute
Estimated remaining time: 14.5246 hours
-------------------------------------------------------------------
Papers 
Processed: 44050
New: 430
Skipped: 43093
Remaining: 59529
Elapsed time: 377.69359135627747 seconds
Proportions: 0.0098 new, 0.9783 skipped
Estimated acquisition rate: 68.31 papers/minute
Estimated remaining time: 14.5244 hours
-------------------------------------------------------------------
Papers 
Processed: 44051
New: 430
Skipped: 43094
Remaining: 59528
Elapsed time: 377.6937310695648 seconds
Proportions: 0.0098 new, 0.9783 skipped
Estimated acquisition rate: 68.31 papers/minute
Estimated remaining time: 14.5241 hours
----------------------------------------------------

xxxxx!!!xxxx!x!xxxxxxxxxx!xx!.xx|
xxxxx!x!!!!!!!!!!!!!!!!!!!!xxxxx.xxxxxxx!xxx!xxxxxx!!!xxxxxx!.!!x!x!xxxxxxxx!!xxxx!!!x!!!x!!!!!x|
xx!!!x.xxxxxxxxxxxxxx!xxxxxxxxxx!xxxx.!!xxxxxxxx!xxxx!xx.xxxxx|
xxxxxxxxxxxxxxxxxxxx.xxxxxxxxxx!xxxxxx!xxxx!!!.!!!!!!!!!!x!!!!!!!!!!!!!!x!!!!!!!.!!!!!x!!!x!.xxx!!!!x!xx|
!xx!!!!!!!!!.!!!x!!!!!!!!!!!!!.!!!!!!!!!!.!!x!x!!!!!!!.x!x!!!!!!xx.xx!!!!!!!.!!!!!!x!!!!.!!!!!!!!!.!!!!!!!!!.!!!!!!!!.!!!!!!!!.!!!!!!!!!!!!!!!!!!!!!!.!!!!!!!!!!!!!!!!!!!!!.!!!!!!!!!!!!!!!!!!!!x!.!x!!!xxx!xxPapers 
Processed: 68841
New: 713
Skipped: 67408
Remaining: 34738
Elapsed time: 550.9760200977325 seconds
Proportions: 0.0104 new, 0.9792 skipped
Estimated acquisition rate: 77.64 papers/minute
Estimated remaining time: 7.4567 hours
-------------------------------------------------------------------
Papers 
Processed: 68842
New: 713
Skipped: 67409
Remaining: 34737
Elapsed time: 550.9764611721039 seconds
Proportions: 0.0104 new, 0.9792 skipped
Estimated acquisition rate: 77.64 papers/minu

Estimated remaining time: 7.4521 hours
-------------------------------------------------------------------
Papers 
Processed: 68864
New: 713
Skipped: 67431
Remaining: 34715
Elapsed time: 550.9886584281921 seconds
Proportions: 0.0104 new, 0.9792 skipped
Estimated acquisition rate: 77.64 papers/minute
Estimated remaining time: 7.4519 hours
-------------------------------------------------------------------
Papers 
Processed: 68865
New: 713
Skipped: 67432
Remaining: 34714
Elapsed time: 550.9899461269379 seconds
Proportions: 0.0104 new, 0.9792 skipped
Estimated acquisition rate: 77.64 papers/minute
Estimated remaining time: 7.4517 hours
-------------------------------------------------------------------
Papers 
Processed: 68866
New: 713
Skipped: 67433
Remaining: 34713
Elapsed time: 550.9902305603027 seconds
Proportions: 0.0104 new, 0.9792 skipped
Estimated acquisition rate: 77.64 papers/minute
Estimated remaining time: 7.4515 hours
----------------------------------------------------------

!xx!xxx.xxxx!x.!!!!!!!.!!!!!!!!!!!.!!!!!!!!!!.!!!x!!!!!!!!!!!!!!.!!!!!!xx!!!.!!!!!xxxx.x!xxxx!xxxx!xxxxx|
xxx!!xx!x!xxx!x!!!!!!!!!!!!!!!!xx!!!!!!!!!!!!!.!!!!!xx!!!!!!xx!xxxxxxx!x!!!!!!!!!!!.!!!!!!!!!!!x!!!!!!xx!!!.!!!!!!!!!!!!!!x!!x!!!x!!!xx.x!x!!xxx!x!!.!!!!!!!!!!.!!!!!!!!!!!!.!!!!!!!!!!!.!!!xx!!!.!!!!!..!!!!!!!!!x!!!!!!!!.!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!.!!x!!x!!!!!x!!!x!x!x!x.!!!!!!!!!xx|
x!!!!!!!!!x!xx!xxxx.xx!!!!!!!!!!!!!!.!!!xxxxx!!.xx!xx!!!xx!!!!!.!!!!!!xx.x!xxxxxx!x!!!x!!!!x.xxxPapers 
Processed: 97450
New: 1068
Skipped: 95545
Remaining: 6129
Elapsed time: 706.0199799537659 seconds
Proportions: 0.011 new, 0.9805 skipped
Estimated acquisition rate: 90.76 papers/minute
Estimated remaining time: 1.1255 hours
-------------------------------------------------------------------
Papers 
Processed: 97451
New: 1068
Skipped: 95546
Remaining: 6128
Elapsed time: 706.0202944278717 seconds
Proportions: 0.011 new, 0.9805 skipped
Estimated acquisition rate: 90.76 papers/minute
Est

xxxxxx!!!.x!!x!x.!x!xxxx|
xxxxxxxxxx.xx!xxxxxxx!xx!x!x.xxx!!!!!!x.x!.!!xxxxx!!!xxxxxxxxxxxxxx!!!!!!x!!!!!!!!!!!!!!!!!!!!!!!!!
\DONE


In [35]:
pdf_url = 'http://arxiv.org/pdf/1704.07057v2.pdf'
req = session.get(pdf_url, timeout=timeout_secs)
req

<Response [200]>

In [36]:
print("{}".format(req.headers))

{'Date': 'Mon, 06 Aug 2018 00:38:35 GMT', 'Server': 'Apache', 'Strict-Transport-Security': 'max-age=31536000', 'Keep-Alive': 'timeout=8, max=100', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'Content-Type': 'text/html; charset=utf-8'}


In [40]:
pdf_url = 'https://arxiv.org/pdf/1706.10044.pdf'
req = session.get(pdf_url, timeout=timeout_secs)
req

<Response [200]>

In [41]:
print("{}".format(req.headers['Content-Type']))

application/pdf


In [42]:
req.headers['Content-Type'] == 'application/pdf'

True